In [9]:
import pandas as pd

In [10]:
data=pd.read_csv("data/pubchem_dump_with_wiki_text.csv")

In [11]:
data.wiki_text.notna().sum()

5635

In [12]:
data

,Unnamed: 0,text,edge,cid,name,wiki_text,combined_text
0,0,\nO-acetylcarnitine is an O-acylcarnitine havi...,"[""7045767"", ""5962"", ""288"", ""6137"", ""176""]",1,Acetyl-DL-carnitine,NaN,wikipedia: \n pubchem:\nO-acetylcarnitine is a...
1,1,NaN,[],2,[2-(Acetyloxy)-3-carboxypropyl]trimethylazanium,NaN,NaN
2,2,"\n2,3-dihydroxy-2,3-dihydrobenzoic acid is a c...","[""23615184"", ""961""]",3,"5,6-Dihydroxycyclohexa-1,3-diene-1-carboxylic ...",NaN,"wikipedia: \n pubchem:\n2,3-dihydroxy-2,3-dihy..."
3,3,\nMonoisopropanolamine appears as a colorless ...,"[""787"", ""222"", ""962"", ""15805271""]",4,1-Aminopropan-2-ol,1-Aminopropan-2-ol is the organic compound wit...,wikipedia: 1-Aminopropan-2-ol is the organic c...
4,4,\n3-amino-2-oxopropyl phosphate is a oxoalkyl ...,[],5,3-Amino-2-oxopropyl phosphate,NaN,wikipedia: \n pubchem:\n3-amino-2-oxopropyl ph...
...,...,...,...,...,...,...,...
49995,49995,"\n2,2',4,5,5'-Pentabromobiphenyl is a polybrom...","[""7095""]",49996,"2,2',4,5,5'-Pentabromobiphenyl",NaN,"wikipedia: \n pubchem:\n2,2',4,5,5'-Pentabromo..."
49996,49996,NaN,[],49997,(S)-Fenpropathrin,NaN,NaN
49997,49997,\nSee also: Methyl dihydroabietate (annotation...,[],49998,"1-Phenanthrenecarboxylic acid, 1,2,3,4,4a,4b,5...",NaN,wikipedia: \n pubchem:\nSee also: Methyl dihyd...
49998,49998,\nHomobaldrinal is a fatty acid ester.\nHomoba...,[],49999,Homobaldrinal,NaN,wikipedia: \n pubchem:\nHomobaldrinal is a fat...
